In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.stats import norm
from scipy.stats import shapiro
from pacal import NormalDistr,UniformDistr

In [ ]:
# Load MNIST Dataset
transform = transforms.Compose(
    [transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

batch_size = 5
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# Definitions

Defining the networks and the Train function

In [ ]:
# network used for investigating neurons strength
class LinearNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 200)
        self.fc2 = nn.Linear(200, 10)
    
    def forward(self, x):
        x = torch.flatten(x, 1)
        self.fc1data = self.fc1(x)
        self.relufc1data = F.relu(self.fc1data)
        self.fc2data = self.fc2(self.relufc1data)
        self.relufc2data = F.relu(self.fc2data)
        return self.fc2data       

# Class for dense 2-layer networks with sigmoid as activation, 
# for mnist or other datasets with input of size 784 and 10 classes

class NetSigmoid2(nn.Module):  
    
    def __init__(self, layer1, layer2):
    #    layer1: number of nodes in the first hidden layer
    #    layer2: number of nodes in second hidden layer
        super().__init__()
        self.fc1 = nn.Linear(784, layer1)
        self.fc2 = nn.Linear(layer1, layer2)
        self.fc3 = nn.Linear(layer2, 10)        
        
    def forward(self, x):
    #   x: input data
    #   return: output of network
        x = torch.flatten(x, 1)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

#Function for training networks, using cross entropy loss
def Train(net, epochs, lr, trainloader, optimizerChoice = "SGD"):
    # net: the net to be trained
    # epochs: number of epochs to train for
    # lr: learning rate/step size for the optimizer
    # trainloader: loader for training data
    # optimizerChoice = name of optimizer, "SGD" (default) or "Adam"
    
    criterion = nn.CrossEntropyLoss()
    
    if optimizerChoice == "SGD":
        optimizer = optim.SGD(net.parameters(), lr=lr)
    elif optimizerChoice == "Adam":
        optimizer = optim.Adam(net.parameters(), lr=lr)
    else:
        raise Exception('Only "SGD" and "Adam" are supported as optimizerChoice')
        
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

This netwok corresponds to the network *net_final.pth*. 
The following code shows how we defined the network and how we trained the networ. It is a very simple network containing an input layer with 28*28 input neurons, one hidden layer with 200 nodes and one output layer with 10 nodes.  

In [ ]:
net = LinearNet()

Train(net,3,0.01,trainloader=trainloader)

In [ ]:
# save model 
PATH = "C:/Users/Catalina/Documents/Bioinformatik/Master/2022_23_WS_Uppsala/Project/net_final.pth"
torch.save(net.state_dict(), PATH)

In [ ]:
# load model
PATH = "C:/Users/Catalina/Documents/Bioinformatik/Master/2022_23_WS_Uppsala/Project/net_final.pth"
net = LinearNet()
net.load_state_dict(torch.load(PATH))

Here we define dense networks with two hidden layers, where the second layer contains 100 nodes for all networks, but the first layer has 2, 10, 100, 1000, or 10000 nodes. Of each type we define and save 10 untrained networks, then train them with sgd and adam as optimizers, to later be able to compare the results depending on optimizer. Code by Lovisa

In [ ]:
sizes = [2, 10, 100, 1000, 10000]
layer2 = 100
epochs = 3
lrSGD = 0.01
lrAdam = 0.001
for i in sizes:
    for k in range(10):
        print(f'network {k} of size {i}')
        net = NetSigmoid2(i, layer2)
        torch.save(net.state_dict(), f'C:\\Users\\lovis\\OneDrive\\Dokument\\SavedNetworks\\Untrained{i}_{layer2}_number{k}')
        Train(net, epochs, lrSGD, trainloader, "SGD")
        torch.save(net.state_dict(), f'C:\\Users\\lovis\\OneDrive\\Dokument\\SavedNetworks\\SGDtrained{i}_{layer2}_number{k}')
        net = NetSigmoid2(i, layer2)
        net.load_state_dict(torch.load(f'C:\\Users\\lovis\\OneDrive\\Dokument\\SavedNetworks\\Untrained{i}_{layer2}_number{k}'))
        net.eval()
        Train(net, epochs, lrAdam, trainloader, "Adam")
        torch.save(net.state_dict(), f'C:\\Users\\lovis\\OneDrive\\Dokument\\SavedNetworks\\Adamtrained{i}_{layer2}_number{k}')

Here starts the training of networks with varying learning rate

In [ ]:
#evaluate accuracy
def evaluate_model(network):
  correct = 0
  total = 0
  with torch.no_grad():
    for i, data in enumerate(testloader,0):
      inputs, labels = data
      outputs = network(inputs)
      for i,e in enumerate(outputs):
        if torch.argmax(e) == labels[i]:
          correct += 1
        total += 1
  return correct/total

#finds the node strength of each node in the first layer
def get_node_strength(network):
  using_bias = network.fc1.bias != None
  if using_bias:
    bias_data = network.fc1.bias

  weight_data = network.fc1.weight.data  
  next_weight_data = network.fc2.weight.data  
  num_nodes = weight_data.size(dim=0) #number of nodes in that layer
  node_strength = np.zeros(num_nodes)  #empty numpy list

  for i in range(num_nodes): #for each node
    node_strength[i] += sum(weight_data[i,:]) #find incoming weights
    if using_bias:
      node_strength[i] += bias_data[i]*weight_data.size(dim=1) #find biases
    node_strength[i] += sum(next_weight_data[:,i]) #find outgoing weights
  return node_strength

In [ ]:
#training many networks to with random learning rate ADAM RELU
num_networks = 100
lrs = np.linspace(0.0001,0.01,num_networks) #varying learning rate
for i in range(num_networks):
  net = LinearNet()
  lr = lrs[i]
  Train(net,3,lr,trainloader,"Adam")
  dt_string = datetime.now().strftime("%H%M%S")
  torch.save(net.state_dict(), f"NetworkProject/lrAnalysis/relu_adam/{dt_string}")
  node_str = get_node_strength(net)
  shap = shapiro(node_str)
  with open('NetworkProject/lrAnalysis/relu_adam/learning_rate_relu_adam_data.csv', 'a',newline="") as f:
    write = csv.writer(f,delimiter=';')
    write.writerow([i,lr,evaluate_model(net),shap[0],shap[1]])

# Plotting the initial distribution of node strength

In [ ]:
n1 = 784
n2 = 10
n = 20000
randomnumbers = []
for i in range(n):
  summa = 0
  w1 = np.random.uniform(-1/np.sqrt(n1),1/np.sqrt(n1),size=n1)
  beta = np.random.uniform(-np.sqrt(n1),np.sqrt(n1))
  w2 = np.random.uniform(-1/np.sqrt(n2),1/np.sqrt(n2),size=n2)

  summa += sum(w1)+beta+sum(w2)
  randomnumbers.append(summa)


plt.figure(figsize=(10,8))
net = LinearNet(num_nodes = n)
node_strength = get_node_strength(net)

plt.hist(node_strength, bins=15, density=True, alpha=0.6, color='g')

mu = np.mean(randomnumbers)
sigma = np.sqrt(np.var(randomnumbers))
print(mu,sigma**2)
N = NormalDistr(0,1/3)
M = NormalDistr(0,1/3)
U = UniformDistr(-np.sqrt(n1),np.sqrt(n1))
Y = N+U+M
Y.plot()
plt.xlabel("Node strength")
plt.ylabel("Probability density")
plt.title(f"Probabiliy density of node strength for a single hidden layer\n in an untrained network with {n} nodes and {n1} input variables.\n Mean = {mu:.2f}, std = {sigma:.2f}")
plt.legend(["Observed","Theoretical"])
plt.savefig("initialdistribution")
plt.show()